In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import seaborn as sns
import time
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/train.csv',index_col='id')
df_test =  pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/test.csv',index_col='id')

In [ ]:
def func_null(df):
    "Statistic the proportion of missing values"
    df_null = pd.DataFrame()
    df_null['count'] = df.isnull().sum()
    df_null['pct'] = df_null['count']/df.shape[0]
    df_null['pct'] = df_null['pct']*100
    df_null = df_null.sort_values(by='count',ascending=False)
    return df_null
func_null(df_train)
#func_null(df_test)

In [ ]:
data_train = df_train.copy()
data_test = df_test.copy()

In [ ]:
quantitative = data_train.columns.tolist()
quantitative.remove('claim')

In [ ]:
data_train['N_missing'] = data_train[quantitative].isnull().sum(axis=1)
data_test['N_missing'] = data_test[quantitative].isnull().sum(axis=1)
quantitative.append('N_missing')

In [ ]:
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.preprocessing import Normalizer,MinMaxScaler,StandardScaler,QuantileTransformer

imp = SimpleImputer(missing_values=np.nan,strategy='constant')
#imp = KNNImputer()
data_train = pd.DataFrame(data=imp.fit_transform(data_train[quantitative]),columns=data_train[quantitative].columns,index=data_train.index)
data_test = pd.DataFrame(data=imp.transform(data_test[quantitative]),columns=data_test[quantitative].columns,index=data_test.index)
preprocessing = StandardScaler()
data_train = pd.DataFrame(data=preprocessing.fit_transform(data_train[quantitative]),columns=data_train[quantitative].columns,index=data_train.index)
data_test = pd.DataFrame(data=preprocessing.transform(data_test[quantitative]),columns=data_test[quantitative].columns,index=data_test.index)


In [ ]:
quantitative.remove('N_missing')
data_train['min_row'] = data_train[quantitative].min(axis=1)
data_train['max_row'] = data_train[quantitative].max(axis=1)
data_train['mean_row'] = data_train[quantitative].mean(axis=1)
data_train['std_row'] = data_train[quantitative].std(axis=1)

data_test['min_row'] = data_test[quantitative].min(axis=1)
data_test['max_row'] = data_test[quantitative].max(axis=1)
data_test['mean_row'] = data_test[quantitative].mean(axis=1)
data_test['std_row'] = data_test[quantitative].std(axis=1)

quantitative = quantitative + ['N_missing','min_row','max_row','mean_row','std_row']

In [ ]:
from sklearn.model_selection import train_test_split,cross_val_score
import lightgbm as lgb

print('start cross-validation !!!')
X_train = data_train[quantitative]
X_test = data_test[quantitative]
y = df_train['claim']

model = lgb.LGBMClassifier()
start=time.time()
t_0 = cross_val_score(model,X_train,y,cv=3,scoring='roc_auc')
print('param_trial_RMSE:%.4f'%(t_0.mean()))
end=time.time()
print('Running time: %s Seconds'%(end-start))

In [ ]:
X_train = data_train[quantitative]
X_test = data_test[quantitative]
y = df_train['claim']

model_1 = lgb.LGBMClassifier()
model_1.fit(X_train,y)
preds_y1 = model_1.predict_proba(X_test)[:,1]
dict_0 = {
    'id':data_test.index.to_numpy(),
    'claim':preds_y1
}
result = pd.DataFrame(dict_0)
result.to_csv('submission.csv',index=False)